# 🔧 Notebook 2: Preprocesamiento y División de Datos

**Objetivo:** Preparar los datos para el modelado mediante codificación, escalamiento y división estratificada.

**Pregunta 4:** Dividir los datos en muestras para entrenamiento y validación (70/30 estratificado)


## 1. Importar Librerías y Cargar Datos


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import warnings

warnings.filterwarnings("ignore")

df = pd.read_excel("../CensoPoblacion.xlsx", sheet_name="adult")
print(f"✅ Dataset cargado: {df.shape}")
df.head()

✅ Dataset cargado: (32561, 10)


,CUSTOMER_ID,EDAD,CAPGANADO,CAPPERD,HORASEMANA,EDUCACIONNUM,EDUCACION,ESTADOCIV,SEXO,INGRESO
0,ID-00PP001,39,2174,0,40,13,Bachelors,Nunca-casado,Masculino,<=50K
1,ID-00PP002,50,0,0,13,13,Bachelors,Casado-civil,Masculino,<=50K
2,ID-00PP003,38,0,0,40,9,HS-grad,Divorciado,Masculino,<=50K
3,ID-00PP004,53,0,0,40,7,11th,Casado-civil,Masculino,<=50K
4,ID-00PP005,28,0,0,40,13,Bachelors,Casado-civil,Femenino,<=50K


## 2. Eliminación de CUSTOMER_ID y Preparación


In [ ]:
df_procesado = df.drop("CUSTOMER_ID", axis=1).copy()
print(f"Dataset sin ID: {df_procesado.shape}")

Dataset sin ID: (32561, 9)


## 3. Codificación de Variable Objetivo (INGRESO)


In [ ]:
le_ingreso = LabelEncoder()
df_procesado["INGRESO_ENCODED"] = le_ingreso.fit_transform(df_procesado["INGRESO"])

print("Mapeo de INGRESO:")
for i, clase in enumerate(le_ingreso.classes_):
    print(f"  {clase} → {i}")

joblib.dump(le_ingreso, "../resultados/encoder_ingreso.pkl")
print("✅ Encoder guardado")

Mapeo de INGRESO:
   <=50K → 0
   >50K → 1
✅ Encoder guardado


## 4. Codificación de Variables Categóricas


In [ ]:
# SEXO: Masculino=1, Femenino=0
df_procesado["SEXO_ENCODED"] = df_procesado["SEXO"].map({"Masculino": 1, "Femenino": 0})
print("Mapeo de SEXO: Masculino=1, Femenino=0")

# One-Hot Encoding para EDUCACION y ESTADOCIV
df_procesado = pd.get_dummies(
    df_procesado, columns=["EDUCACION", "ESTADOCIV"], prefix=["EDU", "ECIV"]
)

print(f"\n✅ Dataset después de encoding: {df_procesado.shape}")
print(f"   Columnas creadas: {df_procesado.shape[1] - df.shape[1]}")

Mapeo de SEXO: Masculino=1, Femenino=0

✅ Dataset después de encoding: (32561, 32)
   Columnas creadas: 22


## 5. Separar Features (X) y Target (y)


In [ ]:
columnas_eliminar = ["INGRESO", "SEXO", "INGRESO_ENCODED"]
X = df_procesado.drop(columnas_eliminar, axis=1)
y = df_procesado["INGRESO_ENCODED"]

print(f"X (features): {X.shape}")
print(f"y (target): {y.shape}")
print(f"\nPrimeras 5 columnas de X: {list(X.columns[:5])}")

X (features): (32561, 29)
y (target): (32561,)

Primeras 5 columnas de X: ['EDAD', 'CAPGANADO', 'CAPPERD', 'HORASEMANA', 'EDUCACIONNUM']


## 6. División Estratificada Train/Test (70/30)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print("=" * 80)
print("DIVISIÓN COMPLETADA")
print("=" * 80)
print(f"Train: {X_train.shape[0]:,} registros ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test:  {X_test.shape[0]:,} registros ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n📊 Balance en TRAIN:")
print((y_train.value_counts(normalize=True) * 100).round(2))
print(f"\n📊 Balance en TEST:")
print((y_test.value_counts(normalize=True) * 100).round(2))
print("\n✅ Estratificación correcta: proporciones mantenidas")

DIVISIÓN COMPLETADA
Train: 22,792 registros (70.0%)
Test:  9,769 registros (30.0%)

📊 Balance en TRAIN:
INGRESO_ENCODED
0    75.92
1    24.08
Name: proportion, dtype: float64

📊 Balance en TEST:
INGRESO_ENCODED
0    75.92
1    24.08
Name: proportion, dtype: float64

✅ Estratificación correcta: proporciones mantenidas


## 7. Escalamiento de Variables


In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, "../resultados/scaler.pkl")

print("✅ Escalamiento completado")
print(f"   Media de features (train): {X_train_scaled.mean():.4f}")
print(f"   Std de features (train): {X_train_scaled.std():.4f}")

✅ Escalamiento completado
   Media de features (train): nan
   Std de features (train): nan


## 8. Guardar Datos Procesados


In [9]:
X_train_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=X.columns)
y_train_df = y_train.reset_index(drop=True)
y_test_df = y_test.reset_index(drop=True)

X_train_df.to_csv("../resultados/X_train.csv", index=False)
X_test_df.to_csv("../resultados/X_test.csv", index=False)
y_train_df.to_csv("../resultados/y_train.csv", index=False, header=["INGRESO_ENCODED"])
y_test_df.to_csv("../resultados/y_test.csv", index=False, header=["INGRESO_ENCODED"])

print("=" * 80)
print("✅ DATOS GUARDADOS EN CARPETA resultados/")
print("=" * 80)
print(f"   - X_train.csv: {X_train_df.shape}")
print(f"   - X_test.csv: {X_test_df.shape}")
print(f"   - y_train.csv: {y_train_df.shape}")
print(f"   - y_test.csv: {y_test_df.shape}")
print(f"   - encoder_ingreso.pkl")
print(f"   - scaler.pkl")
print("\n➡️ Siguiente paso: Notebook 03 - Modelo 1 (Regresión Logística)")

✅ DATOS GUARDADOS EN CARPETA resultados/
   - X_train.csv: (22792, 29)
   - X_test.csv: (9769, 29)
   - y_train.csv: (22792,)
   - y_test.csv: (9769,)
   - encoder_ingreso.pkl
   - scaler.pkl

➡️ Siguiente paso: Notebook 03 - Modelo 1 (Regresión Logística)
